In [17]:
# ===MODULES===
from mlxtend.plotting import scatterplotmatrix
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [3]:
# ===DATA===

train = pd.read_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/train.csv')
test = pd.read_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/test.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/sample_submission.csv')
train.head()

,date_time,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals
0,2018-04-01,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,22994
1,2018-04-02,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,28139
2,2018-04-03,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,26817
3,2018-04-04,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,26034
4,2018-04-05,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,2833


In [4]:
# ===INFO===

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date_time                  273 non-null    object 
 1   wind_direction             273 non-null    float64
 2   sky_condition              273 non-null    float64
 3   precipitation_form         273 non-null    float64
 4   wind_speed                 273 non-null    float64
 5   humidity                   273 non-null    float64
 6   low_temp                   273 non-null    float64
 7   high_temp                  273 non-null    float64
 8   Precipitation_Probability  273 non-null    float64
 9   number_of_rentals          273 non-null    int64  
dtypes: float64(8), int64(1), object(1)
memory usage: 21.5+ KB


In [5]:
# ===UTILS===

# 1. seperate datetime
def seperate_datetime(dataframe):
    year = []
    month = []
    day = []

    for date in dataframe.date_time:
        year_point, month_point, day_point = date.split('-') # - 기준으로 string을 나누고 list로 만듦 ex) '2016-04-01' -> ['2016', '04', '01']
        year.append(int(year_point)-2017)
        month.append(int(month_point)-3)
        day.append(int(day_point))
    return year, month, day


# 2. Calculate NMAE
def NMAE(true, pred):
    score = np.mean(np.abs(true-pred) / true)
    return score



In [7]:
year, month, day = seperate_datetime(train)

train['year'] = year
train['month'] = month
train['day'] = day

X = train.drop(['date_time', 'number_of_rentals'], axis=1)
y = train.number_of_rentals

In [8]:
X_combination = X.copy()

In [10]:
# ===FEATURE ENGINEERING===

# 요일 정보 추가
week_day = pd.to_datetime(train['date_time']).dt.day_name()
le = LabelEncoder()
le.fit(week_day)
X_combination['week_day'] = le.transform(week_day)

# 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
X_combination['temp_diff_info'] = X_combination['high_temp'] - X_combination['low_temp']

# 2. "덥고 습한 날씨"를 알기 위한 정보
X_combination['sweat_info'] = X_combination['high_temp'] * X_combination['humidity'] 

# 3. "춥고 바람부는 날씨"를 알기 위한 정보
X_combination['cold_info'] = X_combination['low_temp'] * X_combination['wind_speed'] 

col_list = X_combination.columns

# 이중 for문을 사용하여 변수 자기 자신의 제곱과 두 변수간의 곱이라는 새로운 변수를 추가합니다.
for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        X_combination[f'{col_list[i]}*{col_list[j]}'] = X_combination[col_list[i]] * X_combination[col_list[j]]

X_combination

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day,week_day,temp_diff_info,sweat_info,cold_info,wind_direction*wind_direction,wind_direction*sky_condition,wind_direction*precipitation_form,wind_direction*wind_speed,wind_direction*humidity,wind_direction*low_temp,wind_direction*high_temp,wind_direction*Precipitation_Probability,wind_direction*year,wind_direction*month,wind_direction*day,wind_direction*week_day,wind_direction*temp_diff_info,wind_direction*sweat_info,wind_direction*cold_info,sky_condition*sky_condition,sky_condition*precipitation_form,sky_condition*wind_speed,sky_condition*humidity,sky_condition*low_temp,sky_condition*high_temp,sky_condition*Precipitation_Probability,sky_condition*year,sky_condition*month,sky_condition*day,...,high_temp*week_day,high_temp*temp_diff_info,high_temp*sweat_info,high_temp*cold_info,Precipitation_Probability*Precipitation_Probability,Precipitation_Probability*year,Precipitation_Probability*month,Precipitation_Probability*day,Precipitation_Probability*week_day,Precipitation_Probability*temp_diff_info,Precipitation_Probability*sweat_info,Precipitation_Probability*cold_info,year*year,year*month,year*day,year*week_day,year*temp_diff_info,year*sweat_info,year*cold_info,month*month,month*day,month*week_day,month*temp_diff_info,month*sweat_info,month*cold_info,day*day,day*week_day,day*temp_diff_info,day*sweat_info,day*cold_info,week_day*week_day,week_day*temp_diff_info,week_day*sweat_info,week_day*cold_info,temp_diff_info*temp_diff_info,temp_diff_info*sweat_info,temp_diff_info*cold_info,sweat_info*sweat_info,sweat_info*cold_info,cold_info*cold_info
0,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,1,1,1,3,8.400,1575.000000,38.430000,43056.250000,830.000000,0.000000,632.875000,15562.500000,2614.500000,4357.500000,6225.000000,207.500,207.500,207.500,622.500,1743.000000,326812.500000,7974.225000,16.000000,0.000000,12.200000,300.000000,50.400000,84.000000,120.000000,4.000,4.000,4.000,...,63.000,176.400000,33075.000000,807.030000,900.000000,30.000,30.000,30.000,90.000,252.000000,47250.000000,1152.900000,1,1,1,3,8.400,1575.000000,38.430000,1,1,3,8.400,1575.000000,38.430000,1,3,8.400,1575.000000,38.430000,9,25.200,4725.000000,115.290000,70.560000,13230.000000,322.812000,2.480625e+06,60527.250000,1476.864900
1,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,1,1,2,1,6.188,1326.827000,41.997736,43395.972489,614.535150,0.000000,682.863126,14547.401061,2668.957404,3958.023000,4062.181500,208.317,208.317,416.634,208.317,1289.065596,276400.620159,8748.842370,8.702500,0.000000,9.670100,206.007350,37.795400,56.050000,57.525000,2.950,2.950,5.900,...,19.000,117.572000,25209.713000,797.956984,380.250000,19.500,19.500,39.000,19.500,120.666000,25873.126500,818.955852,1,1,2,1,6.188,1326.827000,41.997736,1,2,1,6.188,1326.827000,41.997736,4,2,12.376,2653.654000,83.995472,1,6.188,1326.827000,41.997736,38.291344,8210.405476,259.881990,1.760470e+06,55723.730064,1763.809829
2,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,1,1,3,5,5.004,1146.846764,27.739280,45589.082256,621.545076,0.000000,574.358040,15987.864564,2201.776992,3270.211056,4080.931308,213.516,213.516,640.548,1067.580,1068.434064,244870.133662,5922.780108,8.473921,0.000000,7.830590,217.972769,30.018232,44.584876,55.637943,2.911,2.911,8.733,...,76.580,76.641264,17565.105037,424.854812,365.306769,19.113,19.113,57.339,95.565,95.641452,21919.682200,530.180859,1,1,3,5,5.004,1146.846764,27.739280,1,3,5,5.004,1146.846764,27.739280,9,15,15.012,3440.540292,83.217840,25,25.020,5734.233820,138.696400,25.040016,5738.821207,138.807357,1.315258e+06,31812.703504,769.467655
3,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,1,1,4,6,4.056,888.628432,26.083056,20688.794896,531.042512,61.130300,451.357368,10334.472764,1195.564832,1778.963648,6255.859148,143.836,143.836,575.344,863.016,583.398816,127816.759145,3751.682443,13.630864,1.569100,11.585496,265.266508,30.687904,45.662

In [11]:
# ===MODEL1===

forest = RandomForestRegressor(n_estimators=1000,
                               criterion='mse',
                               random_state=1,
                               n_jobs=-1)

forest.fit(X_combination,y)
y_hat = forest.predict(X_combination)

score = NMAE(y, y_hat)

print(f'모델 NMAE: {score}')

모델 NMAE: 0.0873809278749938


In [18]:
# ===MODEL2===

lr = LinearRegression() # 모델 정의
lr.fit(X_combination, y) # 학습
y_hat = lr.predict(X_combination)

score = NMAE(y, y_hat)

print(f'모델 NMAE: {score}')

모델 NMAE: 0.10278949769469102


In [12]:
# ===TEST===

year, month, day = seperate_datetime(test) #날짜 숫자로 분리 후 

#새로운 컬럼을 생성
test['year'] = year 
test['month'] = month
test['day'] = day

# date_time은 제거합니다.
test_X = test.drop('date_time', axis = 1)

# 요일 정보 추가

week_day = pd.to_datetime(test['date_time']).dt.day_name()
le = LabelEncoder()
le.fit(week_day)
test_X['week_day'] = le.transform(week_day)

# 1. "일교차가 너무 큰 날씨"를 알기 위한 정보
test_X['temp_diff_info'] = test_X['high_temp'] - test_X['low_temp']

# 2. "덥고 습한 날씨"를 알기 위한 정보
test_X['sweat_info'] = test_X['high_temp'] * test_X['humidity'] 

# 3. "춥고 바람부는 날씨"를 알기 위한 정보
test_X['cold_info'] = test_X['low_temp'] * test_X['wind_speed'] 

test_X

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day,week_day,temp_diff_info,sweat_info,cold_info
0,108.833,3.000,0.000,2.900,28.333,11.800,20.667,18.333,4,1,1,4,8.867,585.558111,34.220000
1,116.717,3.850,0.000,2.662,46.417,12.000,19.000,28.500,4,1,2,0,7.000,881.923000,31.944000
2,82.669,4.000,0.565,2.165,77.258,8.875,16.368,52.847,4,1,3,2,7.493,1264.558944,19.214375
3,44.123,3.466,0.466,3.747,63.288,6.250,17.368,37.671,4,1,4,3,11.118,1099.185984,23.418750
4,147.791,1.500,0.000,1.560,48.176,7.188,18.684,4.459,4,1,5,1,11.496,900.120384,11.213280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,177.149,3.980,0.223,1.066,74.628,20.312,28.579,36.486,4,3,26,2,8.267,2132.793612,21.652592
87,138.723,2.777,0.135,1.290,70.236,20.812,29.000,18.378,4,3,27,3,8.188,2036.844000,26.847480
88,111.095,3.338,1.270,1.692,70.338,21.000,28.789,35.946,4,3,28,1,7.789,2024.960682,35.532000
89,171.622,3.270,0.595,1.470,70.473,21.000,29.421,27.770,4,3,29,5,8.421,2073.386133,30.870000


In [13]:
col_list = test_X.columns

for i in range(len(col_list)):
    for j in range(i, len(col_list)):
        test_X[f'{col_list[i]}*{col_list[j]}'] = test_X[col_list[i]] * test_X[col_list[j]]

In [16]:
test_X

,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,year,month,day,week_day,temp_diff_info,sweat_info,cold_info,wind_direction*wind_direction,wind_direction*sky_condition,wind_direction*precipitation_form,wind_direction*wind_speed,wind_direction*humidity,wind_direction*low_temp,wind_direction*high_temp,wind_direction*Precipitation_Probability,wind_direction*year,wind_direction*month,wind_direction*day,wind_direction*week_day,wind_direction*temp_diff_info,wind_direction*sweat_info,wind_direction*cold_info,sky_condition*sky_condition,sky_condition*precipitation_form,sky_condition*wind_speed,sky_condition*humidity,sky_condition*low_temp,sky_condition*high_temp,sky_condition*Precipitation_Probability,sky_condition*year,sky_condition*month,sky_condition*day,...,high_temp*week_day,high_temp*temp_diff_info,high_temp*sweat_info,high_temp*cold_info,Precipitation_Probability*Precipitation_Probability,Precipitation_Probability*year,Precipitation_Probability*month,Precipitation_Probability*day,Precipitation_Probability*week_day,Precipitation_Probability*temp_diff_info,Precipitation_Probability*sweat_info,Precipitation_Probability*cold_info,year*year,year*month,year*day,year*week_day,year*temp_diff_info,year*sweat_info,year*cold_info,month*month,month*day,month*week_day,month*temp_diff_info,month*sweat_info,month*cold_info,day*day,day*week_day,day*temp_diff_info,day*sweat_info,day*cold_info,week_day*week_day,week_day*temp_diff_info,week_day*sweat_info,week_day*cold_info,temp_diff_info*temp_diff_info,temp_diff_info*sweat_info,temp_diff_info*cold_info,sweat_info*sweat_info,sweat_info*cold_info,cold_info*cold_info
0,108.833,3.000,0.000,2.900,28.333,11.800,20.667,18.333,4,1,1,4,8.867,585.558111,34.220000,11844.621889,326.499000,0.000000,315.615700,3083.565389,1284.229400,2249.251611,1995.235389,435.332,108.833,108.833,435.332,965.022211,63728.045894,3724.265260,9.000000,0.000000,8.700000,84.999000,35.400000,62.001000,54.999000,12.000,3.000,3.000,...,82.668,183.254289,12101.729480,707.224740,336.098889,73.332,18.333,18.333,73.332,162.558711,10735.036849,627.355260,16,4,4,16,35.468,2342.232444,136.880000,1,1,4,8.867,585.558111,34.220000,1,4,8.867,585.558111,34.220000,16,35.468,2342.232444,136.880000,78.623689,5192.143770,303.428740,3.428783e+05,20037.798558,1171.008400
1,116.717,3.850,0.000,2.662,46.417,12.000,19.000,28.500,4,1,2,0,7.000,881.923000,31.944000,13622.858089,449.360450,0.000000,310.700654,5417.652989,1400.604000,2217.623000,3326.434500,466.868,116.717,233.434,0.000,817.019000,102935.406791,3728.407848,14.822500,0.000000,10.248700,178.705450,46.200000,73.150000,109.725000,15.400,3.850,7.700,...,0.000,133.000000,16756.537000,606.936000,812.250000,114.000,28.500,57.000,0.000,199.500000,25134.805500,910.404000,16,4,8,0,28.000,3527.692000,127.776000,1,2,0,7.000,881.923000,31.944000,4,0,14.000,1763.846000,63.888000,0,0.000,0.000000,0.000000,49.000000,6173.461000,223.608000,7.777882e+05,28172.148312,1020.419136
2,82.669,4.000,0.565,2.165,77.258,8.875,16.368,52.847,4,1,3,2,7.493,1264.558944,19.214375,6834.163561,330.676000,46.707985,178.978385,6386.841602,733.687375,1353.126192,4368.808643,330.676,82.669,248.007,165.338,619.438817,104539.823342,1588.433167,16.000000,2.260000,8.660000,309.032000,35.500000,65.472000,211.388000,16.000,4.000,12.000,...,32.736,122.645424,20698.300795,314.500890,2792.805409,211.388,52.847,158.541,105.694,395.982571,66828.146514,1015.422076,16,4,12,8,29.972,5058.235776,76.857500,1,3,2,7.493,1264.558944,19.214375,9,6,22.479,3793.676832,57.643125,4,14.986,2529.117888,38.428750,56.145049,9475.340167,143.973312,1.599109e+06,24297.709760,369.192207
3,44.123,3.466,0.466,3.747,63.288,6.250,17.368,37.671,4,1,4,3,11.118,1099.185984,23.418750,1946.839129,152.930318,20.561318,165.328881,2792.456424,275.768750,766.328264,1662.157533,176.492,44.123,176.492,132.369,490.559514,48499.383172,1033.305506,12.013156,1.615156,12.987102,219.356208,21.662500,60.197488,130.567

In [15]:
# train data로 학습시킨 모델에 test_X데이터를 넣고 예측합니다.
test_yhat = forest.predict(test_X)

# submission dataFrame 완성
sample_submission['number_of_rentals'] = test_yhat

# 제출 파일 생성
sample_submission.to_csv('/content/drive/MyDrive/gh/kaggle/dacon/bicycle/dataset/FA1.csv', index= False)